# XGB with adaptive log method

In [1]:
"""
COMPREHENSIVE ADAPTIVE MODEL REPORT
====================================

Generate detailed report with:
1. CV metrics (Train R², Test R², Overfitting Gap)
2. Error metrics (RMSE, MAPE, MAE) - CALCULATED DIRECTLY FROM MODEL
3. Feature details per hotel
4. Configuration details
5. Top performers table
6. Complete results table

Format similar to OutofSample-NP-XGB-Iteration2.pdf
"""

import pandas as pd
import numpy as np
import json
from pathlib import Path
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

# ============================================================================
# LOAD RESULTS
# ============================================================================

output_path = Path('../models/adaptive_log_method')

with open(output_path / 'adaptive_summary.json', 'r') as f:
    results = json.load(f)

successful = {h: r for h, r in results.items() if r.get('status') == 'success'}

print("="*80)
print("COMPREHENSIVE ADAPTIVE MODEL REPORT")
print("="*80)
print(f"Based on {len(successful)} successfully trained hotels")
print("="*80)

# ============================================================================
# LOAD DETAILED METRICS
# ============================================================================

# Metric calculation functions
def calculate_mape(y_true, y_pred, epsilon=1e-10):
    """Calculate Mean Absolute Percentage Error"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = np.abs(y_true) > epsilon
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_rmse(y_true, y_pred):
    """Calculate Root Mean Squared Error"""
    return np.sqrt(mean_squared_error(y_true, y_pred))

def calculate_mae(y_true, y_pred):
    """Calculate Mean Absolute Error"""
    return mean_absolute_error(y_true, y_pred)

def detrend_log_series(series, window):
    """Detrend with flexible window"""
    trend = series.rolling(window=window, min_periods=1, center=False).mean()
    detrended = series - trend
    return detrended, trend

def prepare_data_for_metrics(df, config):
    """Prepare data exactly as in training to calculate metrics"""
    df_processed = df.copy()
    feature_cols = []
    
    trend_window = config['trend_window']
    feature_set = config['feature_set']
    
    # Competitor features
    all_comp_lag_cols = [col for col in df.columns 
                         if '_lag_' in col 
                         and 'base_rate' not in col.lower()
                         and any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
    if feature_set == 'simple':
        comp_lag_cols = [col for col in all_comp_lag_cols 
                        if any(f'_lag_{i}' in col for i in [1, 2, 3, 4, 5])]
    else:
        comp_lag_cols = all_comp_lag_cols
    
    # Log + detrend
    for col in comp_lag_cols:
        log_prices = np.log(df_processed[col].replace(0, np.nan))
        detrended, _ = detrend_log_series(log_prices, window=trend_window)
        df_processed[f'{col}_log_detrended'] = detrended
        feature_cols.append(f'{col}_log_detrended')
    
    # Market aggregates (extended only)
    if feature_set == 'extended':
        lag1_cols = [col for col in comp_lag_cols if '_lag_1' in col]
        
        if len(lag1_cols) > 1:
            for col in lag1_cols:
                df_processed[f'{col}_log'] = np.log(df_processed[col].replace(0, np.nan))
            
            lag1_log_cols = [f'{col}_log' for col in lag1_cols]
            
            df_processed['comp_mean_log'] = df_processed[lag1_log_cols].mean(axis=1)
            df_processed['comp_min_log'] = df_processed[lag1_log_cols].min(axis=1)
            df_processed['comp_max_log'] = df_processed[lag1_log_cols].max(axis=1)
            df_processed['comp_std_log'] = df_processed[lag1_log_cols].std(axis=1)
            
            feature_cols.extend(['comp_mean_log', 'comp_min_log', 'comp_max_log', 'comp_std_log'])
    
    # Temporal features
    temporal_cols = ['day_of_week', 'month', 'is_weekend', 'day_of_year']
    temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
    feature_cols.extend(temporal_cols)
    
    if 'day_of_week' in df_processed.columns:
        df_processed['sin_day_of_week'] = np.sin(2 * np.pi * df_processed['day_of_week'] / 7)
        df_processed['cos_day_of_week'] = np.cos(2 * np.pi * df_processed['day_of_week'] / 7)
        feature_cols.extend(['sin_day_of_week', 'cos_day_of_week'])
    
    if 'month' in df_processed.columns:
        df_processed['sin_month'] = np.sin(2 * np.pi * df_processed['month'] / 12)
        df_processed['cos_month'] = np.cos(2 * np.pi * df_processed['month'] / 12)
        feature_cols.extend(['sin_month', 'cos_month'])
    
    if 'day_of_year' in df_processed.columns:
        df_processed['sin_day_of_year'] = np.sin(2 * np.pi * df_processed['day_of_year'] / 365)
        df_processed['cos_day_of_year'] = np.cos(2 * np.pi * df_processed['day_of_year'] / 365)
        feature_cols.extend(['sin_day_of_year', 'cos_day_of_year'])
    
    X = df_processed[feature_cols].copy()
    
    # Target
    y_original = df_processed['base_rate']
    y_log = np.log(y_original.replace(0, np.nan))
    y_detrended, y_trend = detrend_log_series(y_log, window=trend_window)
    
    # Drop NaN
    valid_idx = ~(X.isnull().any(axis=1) | y_detrended.isnull() | y_log.isnull())
    X = X[valid_idx].reset_index(drop=True)
    y_detrended = y_detrended[valid_idx].reset_index(drop=True)
    y_log = y_log[valid_idx].reset_index(drop=True)
    y_trend = y_trend[valid_idx].reset_index(drop=True)
    y_original = y_original[valid_idx].reset_index(drop=True)
    
    return X, y_detrended, y_log, y_trend, y_original, feature_cols

processed_data_path = Path('../data/full-data/processed')
DATA_END_DATE = '2025-12-31'

detailed_results = []

for hotel_id, result in successful.items():
    try:
        # Load model to get feature details
        model_file = output_path / f'{hotel_id}_model.pkl'
        with open(model_file, 'rb') as f:
            model_data = pickle.load(f)
        
        model = model_data['model']
        scaler = model_data['scaler']
        config = model_data['config']
        
        # Load data to calculate additional metrics
        data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        df = pd.read_csv(data_file)
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['date'] <= DATA_END_DATE].copy()
        
        # Get competitor count
        comp_cols = [col for col in df.columns 
                    if '_lag_' in col 
                    and 'base_rate' not in col.lower()
                    and any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
        
        # Count unique competitors (remove lag suffix)
        unique_comps = set()
        for col in comp_cols:
            # Extract competitor name (before _lag_)
            comp_name = col.split('_lag_')[0]
            unique_comps.add(comp_name)
        
        n_competitors = len(unique_comps)
        
        # Calculate additional metrics from data
        base_rates = df['base_rate'].values
        mean_price = float(np.mean(base_rates))
        std_price = float(np.std(base_rates))
        
        # ====================================================================
        # CALCULATE RMSE, MAE, MAPE DIRECTLY FROM MODEL
        # ====================================================================
        
        train_rmse, test_rmse = np.nan, np.nan
        train_mae, test_mae = np.nan, np.nan
        train_mape, test_mape = np.nan, np.nan
        
        try:
            # Prepare data
            X, y_detrended, y_log, y_trend, y_original, feature_cols = prepare_data_for_metrics(df, config)
            
            if len(X) > 0:
                # Scale features
                X_scaled = scaler.transform(X)
                
                # Setup cross-validation
                min_train_size = config.get('min_train_days', 200)
                n_splits = max(2, len(X) // 100)
                tscv = TimeSeriesSplit(n_splits=n_splits)
                
                train_actuals = []
                train_preds = []
                test_actuals = []
                test_preds = []
                
                # Use last CV fold for metrics (same as what was used for R² calculation)
                for train_idx, test_idx in tscv.split(X_scaled):
                    if len(train_idx) < min_train_size:
                        continue
                    
                    # This is the last valid fold
                    X_train_fold = X_scaled[train_idx]
                    X_test_fold = X_scaled[test_idx]
                    y_train_fold = y_detrended.iloc[train_idx]
                    y_test_fold = y_detrended.iloc[test_idx]
                    y_trend_train = y_trend.iloc[train_idx]
                    y_trend_test = y_trend.iloc[test_idx]
                    y_orig_train = y_original.iloc[train_idx]
                    y_orig_test = y_original.iloc[test_idx]
                
                # Make predictions
                train_pred_detrended = model.predict(X_train_fold)
                test_pred_detrended = model.predict(X_test_fold)
                
                # Transform back to original scale
                train_pred_log = train_pred_detrended + y_trend_train.values
                test_pred_log = test_pred_detrended + y_trend_test.values
                
                train_pred_original = np.exp(train_pred_log)
                test_pred_original = np.exp(test_pred_log)
                
                # Calculate metrics
                train_rmse = calculate_rmse(y_orig_train.values, train_pred_original)
                train_mae = calculate_mae(y_orig_train.values, train_pred_original)
                train_mape = calculate_mape(y_orig_train.values, train_pred_original)
                
                test_rmse = calculate_rmse(y_orig_test.values, test_pred_original)
                test_mae = calculate_mae(y_orig_test.values, test_pred_original)
                test_mape = calculate_mape(y_orig_test.values, test_pred_original)
                
        except Exception as e:
            print(f"  Warning: Could not calculate error metrics for {hotel_id}: {e}")
        
        detailed_results.append({
            'hotel_id': hotel_id,
            'observations': len(df),
            'n_features': result['n_features'],
            'n_competitors': n_competitors,
            'train_r2': result['train_r2'],
            'test_r2': result['test_r2'],
            'overfitting_gap': result['train_r2'] - result['test_r2'],
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_mae': train_mae,
            'test_mae': test_mae,
            'train_mape': train_mape,
            'test_mape': test_mape,
            'n_folds': result['n_folds'],
            'trend_window': result['trend_window'],
            'feature_set': result['feature_set'],
            'min_train_days': result['min_train_days'],
            'mean_price': mean_price,
            'std_price': std_price,
            'n_configs_tried': result['n_configs_tried']
        })
        
    except Exception as e:
        print(f"Error processing {hotel_id}: {e}")

df_results = pd.DataFrame(detailed_results)
df_results = df_results.sort_values('test_r2', ascending=False)

# ============================================================================
# SECTION 1: OVERALL PERFORMANCE SUMMARY
# ============================================================================

print(f"\n{'='*80}")
print("1. OVERALL PERFORMANCE SUMMARY")
print(f"{'='*80}")

print(f"\nAggregate Statistics:")
print(f"  Hotels Processed: {len(successful)}")
print(f"  Mean Test R²: {df_results['test_r2'].mean():.4f}")
print(f"  Median Test R²: {df_results['test_r2'].median():.4f}")
print(f"  Std Test R²: {df_results['test_r2'].std():.4f}")
print(f"  Min Test R²: {df_results['test_r2'].min():.4f}")
print(f"  Max Test R²: {df_results['test_r2'].max():.4f}")

print(f"\nTraining Performance:")
print(f"  Mean Train R²: {df_results['train_r2'].mean():.4f}")
print(f"  Median Train R²: {df_results['train_r2'].median():.4f}")

print(f"\nOverfitting Analysis:")
print(f"  Mean Overfitting Gap: {df_results['overfitting_gap'].mean():.4f}")
print(f"  Median Overfitting Gap: {df_results['overfitting_gap'].median():.4f}")
print(f"  Gap Std Dev: {df_results['overfitting_gap'].std():.4f}")

print(f"\nError Metrics:")
print(f"  Mean Test RMSE: ${df_results['test_rmse'].mean():.2f}")
print(f"  Median Test RMSE: ${df_results['test_rmse'].median():.2f}")
print(f"  Mean Test MAE: ${df_results['test_mae'].mean():.2f}")
print(f"  Median Test MAE: ${df_results['test_mae'].median():.2f}")
print(f"  Mean Test MAPE: {df_results['test_mape'].mean():.2f}%")
print(f"  Median Test MAPE: {df_results['test_mape'].median():.2f}%")

print(f"\nComparison to Baseline (22 hotels each):")
print(f"  {'Metric':<25} {'Baseline':<12} {'Adaptive':<12} {'Improvement':<12}")
print(f"  {'-'*60}")
print(f"  {'Mean Test R²':<25} {0.1156:<12.4f} {df_results['test_r2'].mean():<12.4f} {((df_results['test_r2'].mean() - 0.1156) / 0.1156 * 100):>+11.1f}%")
print(f"  {'Median Test R²':<25} {0.2236:<12.4f} {df_results['test_r2'].median():<12.4f} {((df_results['test_r2'].median() - 0.2236) / 0.2236 * 100):>+11.1f}%")
print(f"  {'Max Test R²':<25} {0.6421:<12.4f} {df_results['test_r2'].max():<12.4f} {((df_results['test_r2'].max() - 0.6421) / 0.6421 * 100):>+11.1f}%")

# ============================================================================
# SECTION 2: CONFIGURATION ANALYSIS
# ============================================================================

print(f"\n{'='*80}")
print("2. CONFIGURATION ANALYSIS")
print(f"{'='*80}")

print(f"\nDetrending Window Distribution:")
window_dist = df_results['trend_window'].value_counts().sort_index()
for window, count in window_dist.items():
    pct = count / len(df_results) * 100
    print(f"  {window:2d}-day window: {count:2d} hotels ({pct:5.1f}%)")

print(f"\nFeature Set Distribution:")
feature_dist = df_results['feature_set'].value_counts()
for fset, count in feature_dist.items():
    pct = count / len(df_results) * 100
    print(f"  {fset:10s}: {count:2d} hotels ({pct:5.1f}%)")

print(f"\nMin Training Days Distribution:")
train_dist = df_results['min_train_days'].value_counts().sort_index()
for days, count in train_dist.items():
    pct = count / len(df_results) * 100
    print(f"  {days:3d} days: {count:2d} hotels ({pct:5.1f}%)")

print(f"\nFeature Count Statistics:")
print(f"  Mean: {df_results['n_features'].mean():.1f}")
print(f"  Median: {df_results['n_features'].median():.1f}")
print(f"  Min: {df_results['n_features'].min()}")
print(f"  Max: {df_results['n_features'].max()}")

print(f"\nCompetitor Count Statistics:")
print(f"  Mean: {df_results['n_competitors'].mean():.1f}")
print(f"  Median: {df_results['n_competitors'].median():.1f}")
print(f"  Min: {df_results['n_competitors'].min()}")
print(f"  Max: {df_results['n_competitors'].max()}")

# ============================================================================
# SECTION 3: TOP 10 PERFORMERS
# ============================================================================

print(f"\n{'='*80}")
print("3. TOP 10 PERFORMING HOTELS")
print(f"{'='*80}")
print(f"\n{'Hotel':<12} {'Test R²':<9} {'RMSE':<9} {'MAE':<9} {'MAPE':<9} {'Window':<8} {'Features':<8}")
print(f"{'-'*80}")

for idx, row in df_results.head(10).iterrows():
    print(f"{row['hotel_id']:<12} {row['test_r2']:<9.4f} ${row['test_rmse']:<8.2f} "
          f"${row['test_mae']:<8.2f} {row['test_mape']:<8.2f}% {row['trend_window']:<8} {row['n_features']:<8}")

# ============================================================================
# SECTION 4: COMPLETE RESULTS TABLE (like your PDF)
# ============================================================================

print(f"\n{'='*80}")
print("4. COMPLETE RESULTS TABLE - ALL 22 HOTELS")
print(f"{'='*80}")
print(f"\nTable: Adaptive Log Detrending - Complete Results")
print(f"\n{'Hotel':<12} {'Obs':<5} {'Feat':<5} {'Comps':<6} {'Train R²':<9} {'Test R²':<9} {'Gap':<9} {'RMSE':<9} {'MAE':<9} {'MAPE':<9} {'Window':<7} {'FeatureSet':<10}")
print(f"{'-'*120}")

for idx, row in df_results.iterrows():
    print(f"{row['hotel_id']:<12} {row['observations']:<5} {row['n_features']:<5} "
          f"{row['n_competitors']:<6} {row['train_r2']:<9.4f} {row['test_r2']:<9.4f} "
          f"{row['overfitting_gap']:<9.4f} ${row['test_rmse']:<8.2f} ${row['test_mae']:<8.2f} "
          f"{row['test_mape']:<8.2f}% {row['trend_window']:<7} {row['feature_set']:<10}")

# ============================================================================
# SECTION 5: FEATURE DETAILS FOR TOP 5 HOTELS
# ============================================================================

print(f"\n{'='*80}")
print("5. DETAILED FEATURE BREAKDOWN - TOP 5 HOTELS")
print(f"{'='*80}")

for idx, row in df_results.head(5).iterrows():
    hotel_id = row['hotel_id']
    
    print(f"\n{hotel_id}: {row['n_features']} total features")
    print(f"  Configuration:")
    print(f"    - Detrending window: {row['trend_window']} days")
    print(f"    - Feature set: {row['feature_set']}")
    print(f"    - Min training: {row['min_train_days']} days")
    print(f"    - Competitors: {row['n_competitors']}")
    print(f"  Performance:")
    print(f"    - Test R²: {row['test_r2']:.4f}")
    print(f"    - Train R²: {row['train_r2']:.4f}")
    print(f"    - Overfitting gap: {row['overfitting_gap']:.4f}")
    print(f"    - Test RMSE: ${row['test_rmse']:.2f}")
    print(f"    - Test MAE: ${row['test_mae']:.2f}")
    print(f"    - Test MAPE: {row['test_mape']:.2f}%")
    print(f"  Data:")
    print(f"    - Observations: {row['observations']}")
    print(f"    - CV folds: {row['n_folds']}")
    print(f"    - Configs tried: {row['n_configs_tried']}")
    
    # Load model to show feature names
    try:
        model_file = output_path / f'{hotel_id}_model.pkl'
        with open(model_file, 'rb') as f:
            model_data = pickle.load(f)
        
        feature_cols = model_data['feature_cols']
        
        # Categorize features
        comp_features = [f for f in feature_cols if 'lag' in f.lower() and 'detrended' in f]
        market_features = [f for f in feature_cols if 'comp_' in f and 'detrended' not in f]
        temporal_features = [f for f in feature_cols if f not in comp_features + market_features]
        
        print(f"  Feature breakdown:")
        print(f"    - Competitor lag features: {len(comp_features)}")
        print(f"    - Market aggregate features: {len(market_features)}")
        print(f"    - Temporal features: {len(temporal_features)}")
        
        if row['feature_set'] == 'simple':
            print(f"    - Using lags 1-5 only (simple)")
        else:
            print(f"    - Using all available lags (extended)")
        
    except:
        pass

# ============================================================================
# SECTION 6: PERFORMANCE DISTRIBUTION
# ============================================================================

print(f"\n{'='*80}")
print("6. PERFORMANCE DISTRIBUTION")
print(f"{'='*80}")

bins = [
    (float('-inf'), 0, 'Negative', '❌'),
    (0, 0.2, 'Low (0-0.2)', '✓'),
    (0.2, 0.4, 'Medium (0.2-0.4)', '⭐'),
    (0.4, 0.6, 'High (0.4-0.6)', '🌟'),
    (0.6, float('inf'), 'Very High (>0.6)', '🏆')
]

print(f"\n{'Tier':<25} {'Count':<8} {'Percentage':<12} {'Hotels'}")
print(f"{'-'*80}")

for low, high, label, emoji in bins:
    if high == float('inf'):
        hotels_in_bin = df_results[df_results['test_r2'] > low]
    elif low == float('-inf'):
        hotels_in_bin = df_results[df_results['test_r2'] <= high]
    else:
        hotels_in_bin = df_results[(df_results['test_r2'] > low) & (df_results['test_r2'] <= high)]
    
    count = len(hotels_in_bin)
    pct = count / len(df_results) * 100
    hotel_ids = ', '.join(hotels_in_bin['hotel_id'].tolist()[:5])
    if count > 5:
        hotel_ids += f", ... ({count-5} more)"
    
    print(f"{emoji} {label:<22} {count:<8} {pct:>5.1f}%        {hotel_ids}")

# ============================================================================
# SECTION 7: KEY INSIGHTS
# ============================================================================

print(f"\n{'='*80}")
print("7. KEY INSIGHTS")
print(f"{'='*80}")

print(f"\n✅ SUCCESS FACTORS:")
print(f"  1. 7-day detrending window works best for ALL hotels")
print(f"     - Short window = better captures recent price changes")
print(f"     - More responsive to market dynamics")

simple_r2 = df_results[df_results['feature_set'] == 'simple']['test_r2'].mean()
extended_r2 = df_results[df_results['feature_set'] == 'extended']['test_r2'].mean()

print(f"\n  2. Simple features (lags 1-5) generally outperform extended")
print(f"     - Simple mean R²: {simple_r2:.4f}")
print(f"     - Extended mean R²: {extended_r2:.4f}")
print(f"     - 16/22 hotels (72.7%) chose simple features")

print(f"\n  3. Adaptive approach achieved excellent results:")
print(f"     - +303% mean improvement vs baseline")
print(f"     - +98% median improvement vs baseline")
print(f"     - 21/22 hotels with R² > 0 (95.5% success rate)")
print(f"     - Mean Test MAPE: {df_results['test_mape'].mean():.2f}%")
print(f"     - Mean Test RMSE: ${df_results['test_rmse'].mean():.2f}")

print(f"\n  4. Coverage matches baseline:")
print(f"     - Both methods: 22 hotels")
print(f"     - But adaptive has MUCH better performance")

print(f"\n⚠️  AREAS FOR IMPROVEMENT:")
print(f"  1. One hotel (Hotel_24) still has negative R²")
print(f"     - May need special handling or fallback")

gap_high = df_results[df_results['overfitting_gap'] > 0.5]
print(f"\n  2. {len(gap_high)} hotels have overfitting gap > 0.5")
print(f"     - Mean gap: {df_results['overfitting_gap'].mean():.4f}")
print(f"     - May benefit from stronger regularization")

# ============================================================================
# SECTION 8: SAVE COMPREHENSIVE REPORT
# ============================================================================

print(f"\n{'='*80}")
print("8. SAVING COMPREHENSIVE REPORT")
print(f"{'='*80}")

# Save detailed CSV
df_results.to_csv(output_path / 'comprehensive_report.csv', index=False)
print(f"\n✅ Saved: comprehensive_report.csv")

# Save summary statistics
summary_stats = {
    'total_hotels': len(successful),
    'mean_test_r2': float(df_results['test_r2'].mean()),
    'median_test_r2': float(df_results['test_r2'].median()),
    'mean_train_r2': float(df_results['train_r2'].mean()),
    'mean_overfitting_gap': float(df_results['overfitting_gap'].mean()),
    'mean_test_rmse': float(df_results['test_rmse'].mean()),
    'median_test_rmse': float(df_results['test_rmse'].median()),
    'mean_test_mae': float(df_results['test_mae'].mean()),
    'median_test_mae': float(df_results['test_mae'].median()),
    'mean_test_mape': float(df_results['test_mape'].mean()),
    'median_test_mape': float(df_results['test_mape'].median()),
    'baseline_comparison': {
        'baseline_mean_r2': 0.1156,
        'baseline_median_r2': 0.2236,
        'improvement_mean_pct': float((df_results['test_r2'].mean() - 0.1156) / 0.1156 * 100),
        'improvement_median_pct': float((df_results['test_r2'].median() - 0.2236) / 0.2236 * 100)
    },
    'configuration_insights': {
        'most_common_window': int(df_results['trend_window'].mode()[0]),
        'most_common_feature_set': str(df_results['feature_set'].mode()[0]),
        'mean_features': float(df_results['n_features'].mean()),
        'mean_competitors': float(df_results['n_competitors'].mean())
    }
}

with open(output_path / 'summary_statistics.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)
print(f"✅ Saved: summary_statistics.json")

# Create deployment guide
deployment_guide = df_results[[
    'hotel_id', 'test_r2', 'test_rmse', 'test_mae', 'test_mape',
    'trend_window', 'feature_set', 
    'n_features', 'n_competitors', 'n_folds'
]].copy()
deployment_guide['status'] = deployment_guide['test_r2'].apply(
    lambda x: 'DEPLOY' if x > 0.2 else ('REVIEW' if x > 0 else 'FALLBACK')
)
deployment_guide.to_csv(output_path / 'deployment_guide.csv', index=False)
print(f"✅ Saved: deployment_guide.csv")

print(f"\n{'='*80}")
print("🎉 COMPREHENSIVE REPORT COMPLETE!")
print(f"{'='*80}")
print(f"\nGenerated files:")
print(f"  📊 comprehensive_report.csv - Full details for all hotels (WITH RMSE, MAE, MAPE)")
print(f"  📈 summary_statistics.json - Aggregate statistics (WITH ERROR METRICS)")
print(f"  🚀 deployment_guide.csv - Deployment recommendations (WITH ERROR METRICS)")
print(f"\nLocation: {output_path}/")
print(f"\n{'='*80}")

COMPREHENSIVE ADAPTIVE MODEL REPORT
Based on 22 successfully trained hotels

1. OVERALL PERFORMANCE SUMMARY

Aggregate Statistics:
  Hotels Processed: 22
  Mean Test R²: 0.4659
  Median Test R²: 0.4426
  Std Test R²: 0.2469
  Min Test R²: -0.2555
  Max Test R²: 0.7890

Training Performance:
  Mean Train R²: 0.7947
  Median Train R²: 0.7841

Overfitting Analysis:
  Mean Overfitting Gap: 0.3288
  Median Overfitting Gap: 0.3087
  Gap Std Dev: 0.2563

Error Metrics:
  Mean Test RMSE: $25.65
  Median Test RMSE: $20.57
  Mean Test MAE: $16.65
  Median Test MAE: $13.83
  Mean Test MAPE: 6.43%
  Median Test MAPE: 6.57%

Comparison to Baseline (22 hotels each):
  Metric                    Baseline     Adaptive     Improvement 
  ------------------------------------------------------------
  Mean Test R²              0.1156       0.4659            +303.0%
  Median Test R²            0.2236       0.4426             +97.9%
  Max Test R²               0.6421       0.7890             +22.9%

2. CONF

## Detailed Feature analysis

In [2]:

import pandas as pd
import numpy as np
import json
from pathlib import Path
import pickle
import shap
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# SETUP
# ============================================================================

output_path = Path('../models/adaptive_log_method')
processed_data_path = Path('../data/full-data/processed')

with open(output_path / 'adaptive_summary.json', 'r') as f:
    results = json.load(f)

successful = {h: r for h, r in results.items() if r.get('status') == 'success'}

print("="*80)
print("DETAILED FEATURE ANALYSIS WITH SHAP")
print("="*80)
print(f"Analyzing {len(successful)} hotels")
print("="*80)

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def detrend_log_series(series, window):
    """Detrend with flexible window"""
    trend = series.rolling(window=window, min_periods=1, center=False).mean()
    detrended = series - trend
    return detrended, trend

def prepare_features_for_shap(df, config):
    """Prepare features exactly as in training"""
    df_processed = df.copy()
    feature_cols = []
    
    trend_window = config['trend_window']
    feature_set = config['feature_set']
    
    # Competitor features
    all_comp_lag_cols = [col for col in df.columns 
                         if '_lag_' in col 
                         and 'base_rate' not in col.lower()
                         and any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
    if feature_set == 'simple':
        comp_lag_cols = [col for col in all_comp_lag_cols 
                        if any(f'_lag_{i}' in col for i in [1, 2, 3, 4, 5])]
    else:
        comp_lag_cols = all_comp_lag_cols
    
    # Log + detrend
    for col in comp_lag_cols:
        log_prices = np.log(df_processed[col].replace(0, np.nan))
        detrended, _ = detrend_log_series(log_prices, window=trend_window)
        df_processed[f'{col}_log_detrended'] = detrended
        feature_cols.append(f'{col}_log_detrended')
    
    # Market aggregates (extended only)
    if feature_set == 'extended':
        lag1_cols = [col for col in comp_lag_cols if '_lag_1' in col]
        
        if len(lag1_cols) > 1:
            for col in lag1_cols:
                df_processed[f'{col}_log'] = np.log(df_processed[col].replace(0, np.nan))
            
            lag1_log_cols = [f'{col}_log' for col in lag1_cols]
            
            df_processed['comp_mean_log'] = df_processed[lag1_log_cols].mean(axis=1)
            df_processed['comp_min_log'] = df_processed[lag1_log_cols].min(axis=1)
            df_processed['comp_max_log'] = df_processed[lag1_log_cols].max(axis=1)
            df_processed['comp_std_log'] = df_processed[lag1_log_cols].std(axis=1)
            
            feature_cols.extend(['comp_mean_log', 'comp_min_log', 'comp_max_log', 'comp_std_log'])
    
    # Temporal features
    temporal_cols = ['day_of_week', 'month', 'is_weekend', 'day_of_year']
    temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
    feature_cols.extend(temporal_cols)
    
    if 'day_of_week' in df_processed.columns:
        df_processed['sin_day_of_week'] = np.sin(2 * np.pi * df_processed['day_of_week'] / 7)
        df_processed['cos_day_of_week'] = np.cos(2 * np.pi * df_processed['day_of_week'] / 7)
        feature_cols.extend(['sin_day_of_week', 'cos_day_of_week'])
    
    if 'month' in df_processed.columns:
        df_processed['sin_month'] = np.sin(2 * np.pi * df_processed['month'] / 12)
        df_processed['cos_month'] = np.cos(2 * np.pi * df_processed['month'] / 12)
        feature_cols.extend(['sin_month', 'cos_month'])
    
    if 'day_of_year' in df_processed.columns:
        df_processed['sin_day_of_year'] = np.sin(2 * np.pi * df_processed['day_of_year'] / 365)
        df_processed['cos_day_of_year'] = np.cos(2 * np.pi * df_processed['day_of_year'] / 365)
        feature_cols.extend(['sin_day_of_year', 'cos_day_of_year'])
    
    X = df_processed[feature_cols].copy()
    
    # Target
    y_original = df_processed['base_rate']
    y_log = np.log(y_original.replace(0, np.nan))
    y_detrended, y_trend = detrend_log_series(y_log, window=trend_window)
    
    # Drop NaN
    valid_idx = ~(X.isnull().any(axis=1) | y_detrended.isnull() | y_log.isnull())
    X = X[valid_idx].reset_index(drop=True)
    
    return X, feature_cols

def categorize_features(feature_cols):
    """Categorize features into groups"""
    categories = {
        'competitor_lags': [],
        'market_aggregates': [],
        'temporal_basic': [],
        'temporal_cyclical': []
    }
    
    for feat in feature_cols:
        if 'lag' in feat.lower() and 'detrended' in feat:
            categories['competitor_lags'].append(feat)
        elif 'comp_' in feat and 'detrended' not in feat:
            categories['market_aggregates'].append(feat)
        elif 'sin_' in feat or 'cos_' in feat:
            categories['temporal_cyclical'].append(feat)
        else:
            categories['temporal_basic'].append(feat)
    
    return categories

# ============================================================================
# ANALYZE EACH HOTEL
# ============================================================================

all_hotel_features = []

# Process top 10 hotels for detailed analysis
results_sorted = sorted(successful.items(), key=lambda x: x[1]['test_r2'], reverse=True)

for idx, (hotel_id, result) in enumerate(results_sorted, 1):
    print(f"\n{'='*80}")
    print(f"HOTEL {idx}/22: {hotel_id}")
    print(f"{'='*80}")
    
    try:
        # Load model
        model_file = output_path / f'{hotel_id}_model.pkl'
        with open(model_file, 'rb') as f:
            model_data = pickle.load(f)
        
        model = model_data['model']
        scaler = model_data['scaler']
        feature_cols = model_data['feature_cols']
        config = model_data['config']
        
        # Load data
        data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        df = pd.read_csv(data_file)
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['date'] <= DATA_END_DATE].copy()
        
        # Prepare features
        X, _ = prepare_features_for_shap(df, config)
        
        # Scale
        X_scaled = scaler.transform(X)
        
        # Calculate SHAP values
        print(f"\nCalculating SHAP values...")
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_scaled[:100])  # Use first 100 samples for speed
        
        # Get mean absolute SHAP values
        mean_abs_shap = np.abs(shap_values).mean(axis=0)
        
        # Create feature importance dataframe
        feature_importance = pd.DataFrame({
            'feature': feature_cols,
            'shap_importance': mean_abs_shap,
            'xgb_importance': model.feature_importances_
        })
        feature_importance = feature_importance.sort_values('shap_importance', ascending=False)
        
        # Categorize features
        categories = categorize_features(feature_cols)
        
        # Print summary
        print(f"\nConfiguration:")
        print(f"  Detrending window: {config['trend_window']} days")
        print(f"  Feature set: {config['feature_set']}")
        print(f"  Min training: {config['min_train_days']} days")
        
        print(f"\nPerformance:")
        print(f"  Test R2: {result['test_r2']:.4f}")
        print(f"  Train R2: {result['train_r2']:.4f}")
        print(f"  Overfitting gap: {result['train_r2'] - result['test_r2']:.4f}")
        
        print(f"\nFeature Categories:")
        print(f"  Competitor lags: {len(categories['competitor_lags'])}")
        print(f"  Market aggregates: {len(categories['market_aggregates'])}")
        print(f"  Temporal basic: {len(categories['temporal_basic'])}")
        print(f"  Temporal cyclical: {len(categories['temporal_cyclical'])}")
        print(f"  Total features: {len(feature_cols)}")
        
        print(f"\nTop 10 Features by SHAP Importance:")
        print(f"  Rank  Feature                                            SHAP          XGB")
        print(f"  --------------------------------------------------------------------------------")
        for i, row in feature_importance.head(10).iterrows():
            print(f"  {feature_importance.index.get_loc(i)+1:<5} {row['feature']:<50} "
                  f"{row['shap_importance']:<12.6f} {row['xgb_importance']:<12.6f}")
        
        # Store for summary
        all_hotel_features.append({
            'hotel_id': hotel_id,
            'test_r2': result['test_r2'],
            'n_features': len(feature_cols),
            'n_competitor_lags': len(categories['competitor_lags']),
            'n_market_agg': len(categories['market_aggregates']),
            'n_temporal': len(categories['temporal_basic']) + len(categories['temporal_cyclical']),
            'trend_window': config['trend_window'],
            'feature_set': config['feature_set'],
            'top_feature': feature_importance.iloc[0]['feature'],
            'top_feature_shap': feature_importance.iloc[0]['shap_importance'],
            'all_features': feature_cols,
            'feature_importance': feature_importance.to_dict('records')
        })
        
        # Save detailed feature list for this hotel
        feature_importance.to_csv(
            output_path / f'{hotel_id}_feature_importance.csv', 
            index=False
        )
        print(f"\nSaved: {hotel_id}_feature_importance.csv")
        
        # Print exact feature list (for top 5 hotels only)
        if idx <= 5:
            print(f"\nEXACT FEATURE LIST (All {len(feature_cols)} features):")
            print(f"\nCompetitor Lag Features ({len(categories['competitor_lags'])}):")
            for feat in categories['competitor_lags']:
                print(f"  - {feat}")
            
            if len(categories['market_aggregates']) > 0:
                print(f"\nMarket Aggregate Features ({len(categories['market_aggregates'])}):")
                for feat in categories['market_aggregates']:
                    print(f"  - {feat}")
            
            print(f"\nTemporal Basic Features ({len(categories['temporal_basic'])}):")
            for feat in categories['temporal_basic']:
                print(f"  - {feat}")
            
            print(f"\nTemporal Cyclical Features ({len(categories['temporal_cyclical'])}):")
            for feat in categories['temporal_cyclical']:
                print(f"  - {feat}")
        
    except Exception as e:
        print(f"Error analyzing {hotel_id}: {e}")
        import traceback
        traceback.print_exc()

# ============================================================================
# SUMMARY ANALYSIS
# ============================================================================

print(f"\n{'='*80}")
print("SUMMARY: FEATURE IMPORTANCE PATTERNS")
print(f"{'='*80}")

df_features = pd.DataFrame(all_hotel_features)

print(f"\nAverage Feature Counts:")
print(f"  Total features: {df_features['n_features'].mean():.1f} (std {df_features['n_features'].std():.1f})")
print(f"  Competitor lags: {df_features['n_competitor_lags'].mean():.1f} (std {df_features['n_competitor_lags'].std():.1f})")
print(f"  Market aggregates: {df_features['n_market_agg'].mean():.1f} (std {df_features['n_market_agg'].std():.1f})")
print(f"  Temporal: {df_features['n_temporal'].mean():.1f} (std {df_features['n_temporal'].std():.1f})")

print(f"\nMost Important Features (Top feature per hotel):")
top_features = df_features['top_feature'].value_counts().head(10)
for feat, count in top_features.items():
    print(f"  {feat}: {count} hotels")

print(f"\nCorrelation: Feature Count vs Performance")
corr = df_features[['n_features', 'test_r2']].corr().iloc[0, 1]
print(f"  Correlation(n_features, test_r2): {corr:.3f}")

# Save complete feature analysis
df_features_export = df_features.drop(['all_features', 'feature_importance'], axis=1)
df_features_export.to_csv(output_path / 'feature_analysis_summary.csv', index=False)

# Save complete feature lists
with open(output_path / 'complete_feature_lists.json', 'w') as f:
    feature_lists = {
        hotel['hotel_id']: {
            'features': hotel['all_features'],
            'n_features': hotel['n_features'],
            'test_r2': hotel['test_r2'],
            'config': {
                'trend_window': hotel['trend_window'],
                'feature_set': hotel['feature_set']
            }
        }
        for hotel in all_hotel_features
    }
    json.dump(feature_lists, f, indent=2)

print(f"\n{'='*80}")
print("FEATURE ANALYSIS COMPLETE!")
print(f"{'='*80}")
print(f"\nGenerated files:")
print(f"  feature_analysis_summary.csv - Summary for all hotels")
print(f"  complete_feature_lists.json - Exact features per hotel")
print(f"  [hotel_id]_feature_importance.csv - SHAP values per hotel")
print(f"\nLocation: {output_path}/")
print(f"{'='*80}")

DETAILED FEATURE ANALYSIS WITH SHAP
Analyzing 22 hotels

HOTEL 1/22: Hotel_22

Calculating SHAP values...

Configuration:
  Detrending window: 7 days
  Feature set: simple
  Min training: 250 days

Performance:
  Test R2: 0.7890
  Train R2: 0.7767
  Overfitting gap: -0.0123

Feature Categories:
  Competitor lags: 45
  Market aggregates: 0
  Temporal basic: 4
  Temporal cyclical: 6
  Total features: 55

Top 10 Features by SHAP Importance:
  Rank  Feature                                            SHAP          XGB
  --------------------------------------------------------------------------------
  1     day_of_week                                        0.047595     0.572065    
  2     sin_day_of_week                                    0.000699     0.186473    
  3     cos_day_of_year                                    0.000453     0.120606    
  4     cos_month                                          0.000408     0.120856    
  5     booking-us-ac-by-marriott-san-rafael-USD_lag_1_log


Calculating SHAP values...

Configuration:
  Detrending window: 7 days
  Feature set: extended
  Min training: 250 days

Performance:
  Test R2: 0.7105
  Train R2: 0.5893
  Overfitting gap: -0.1212

Feature Categories:
  Competitor lags: 30
  Market aggregates: 4
  Temporal basic: 4
  Temporal cyclical: 6
  Total features: 44

Top 10 Features by SHAP Importance:
  Rank  Feature                                            SHAP          XGB
  --------------------------------------------------------------------------------
  1     sin_day_of_week                                    0.040420     0.239863    
  2     cos_day_of_week                                    0.018818     0.080246    
  3     day_of_week                                        0.015450     0.083131    
  4     cos_day_of_year                                    0.005636     0.031263    
  5     booking-us-hyatt-place-fredericksburg-at-mary-washington-USD_lag_5_log_detrended 0.005039     0.064535    
  6     comp_min_lo


Configuration:
  Detrending window: 7 days
  Feature set: simple
  Min training: 200 days

Performance:
  Test R2: 0.6609
  Train R2: 0.8338
  Overfitting gap: 0.1729

Feature Categories:
  Competitor lags: 30
  Market aggregates: 0
  Temporal basic: 4
  Temporal cyclical: 6
  Total features: 40

Top 10 Features by SHAP Importance:
  Rank  Feature                                            SHAP          XGB
  --------------------------------------------------------------------------------
  1     day_of_week                                        0.084363     0.184418    
  2     booking-us-triada-palm-springs-autograph-collection-USD_lag_1_log_detrended 0.021797     0.024832    
  3     booking-us-ace-and-swim-club-USD_lag_4_log_detrended 0.021539     0.076134    
  4     cos_day_of_week                                    0.020519     0.026876    
  5     booking-us-palm-desert-1620-south-indian-trail-USD_lag_1_log_detrended 0.020239     0.028190    
  6     booking-us-ace-and-swim-c


Calculating SHAP values...

Configuration:
  Detrending window: 7 days
  Feature set: simple
  Min training: 250 days

Performance:
  Test R2: 0.4395
  Train R2: 0.9309
  Overfitting gap: 0.4914

Feature Categories:
  Competitor lags: 45
  Market aggregates: 0
  Temporal basic: 4
  Temporal cyclical: 6
  Total features: 55

Top 10 Features by SHAP Importance:
  Rank  Feature                                            SHAP          XGB
  --------------------------------------------------------------------------------
  1     booking-us-painted-buffalo-inn-USD_lag_1_log_detrended 0.027018     0.070381    
  2     day_of_week                                        0.026215     0.126600    
  3     cos_day_of_week                                    0.024434     0.058442    
  4     day_of_year                                        0.012168     0.059504    
  5     booking-us-virginian-lodge-USD_lag_1_log_detrended 0.006219     0.045557    
  6     sin_day_of_year                         


Calculating SHAP values...

Configuration:
  Detrending window: 7 days
  Feature set: extended
  Min training: 200 days

Performance:
  Test R2: 0.2967
  Train R2: 0.7528
  Overfitting gap: 0.4561

Feature Categories:
  Competitor lags: 15
  Market aggregates: 4
  Temporal basic: 4
  Temporal cyclical: 6
  Total features: 29

Top 10 Features by SHAP Importance:
  Rank  Feature                                            SHAP          XGB
  --------------------------------------------------------------------------------
  1     day_of_week                                        0.055285     0.214093    
  2     sin_day_of_week                                    0.006772     0.102162    
  3     booking-fr-ibis-styles-paris-boulogne-marcel-sembat-EUR_lag_4_log_detrended 0.006676     0.041764    
  4     booking-fr-ibis-styles-paris-boulogne-marcel-sembat-EUR_lag_1_log_detrended 0.006662     0.039020    
  5     booking-fr-villa-sorel-EUR_lag_4_log_detrended     0.006593     0.046562    
